In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

## Define our derivatives

In [ ]:
def dydx(x,y):
    y_derivs = np.zeros(2) #creating a 2x1 array to house derivative values
    y_derivs[0] = y[1] #setting the first element to y[1]
    y_derivs[1] = -1*y[0] #setting the second element to the negative of y[0]
    
    return y_derivs

## Define 4th order RK method

In [ ]:
def rk4_mv_core(dydx, xi, yi, nv, h): #defining the core function of our 4th order method
    
    k1 = np.zeros(nv) #creating 4 different arrays each of size nv, the length of yi
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    
    x_ipoh = xi + 0.5*h #generating step functions for half and full steps 
    x_ipo = xi + h
    
    y_temp = np.zeros(nv) #creating a y_temp array
    y_derivs = dydx(xi,yi) #filling k arrays with values based off of derivative steps
    k1[:] = h*y_derivs[:]
    
    
    y_temp[:] = yi[:] + 0.5*k1[:] #filling k2 array with values
    y_derivs = dydx(x_ipoh,y_temp)
    k2[:] = h*y_derivs[:]
    
    y_temp[:] = yi[:] + 0.5*k2[:] #getting k3 values
    y_derivs = dydx(x_ipoh,y_temp)
    k3[:] = h*y_derivs[:]
    
    y_temp[:] = yi[:] + k3[:] #finishing by getting k4 values
    y_derivs = dydx(x_ipo, y_temp)
    k4[:] = h*y_derivs[:]
    
    yipo = yi + (k1 + 2*k2 + 2*k3 + k4)/6. #function to bring y forwards by an h sized step
    
    return yipo #returning advanced y value
    

## Define a step size driver

In [ ]:
def rk4_mv_ad(dydx, x_i, y_i, nv, h, tol): #defining a step driver
    SAFETY = 0.9 #creating a safety variable incase higher valued attempts fall outside of our error
    H_NEW_FAC = 2.0 #creating a reach value that we could get lucky with
    
    imax = 10000 #max iteration value
    
    i = 0 #setting a variable to store iterations
    
    Delta = np.full(nv, 2*tol) #creating an error function
    
    h_step = h #step storage variable
    
    while(Delta.max()/tol > 1.0): #while loop to test our two different types of steps
        
        y_2 = rk4_mv_core(dydx, x_i, y_i, nv, h_step) #full step function
        y_1 = rk4_mv_core(dydx, x_i, y_i, nv, 0.5*h_step) #two half step functions
        y_11 = rk4_mv_core(dydx, x_i+0.5*h_step, y_1, nv, 0.5*h_step)
        
        Delta = np.fabs(y_2 - y_11) #figuring out the difference between both types of tests
        
        if(Delta.max()/tol>1.0): #checking to see if the difference is too big to continue with
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25) #if it is then we use our safety variable to decrease the step
            
        if(i>=imax): #setting up a response for over iterating
            print("Too many steps")
            raise StopIteration("Ending after: ",i)
            
        i += 1
        
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC) #creating a variable to attempt a bigger step
    
    return y_2, h_new, h_step #returning y_2 (our answer), h_new (a next attempt at a bigger step), and h_step (our actual step)


In [ ]:
def rk4_mv(dydx, a, b, y_a, tol): #wrapper for rk4
    
    xi = a #variable for our starting value
    yi = y_a.copy() #boundary condition
    
    h = 1.0e-4*(b-a) #setting up an initial step size
    
    imax = 10000
    
    i = 0
    
    nv = len(y_a) #setting the size of our coupled nodes equal to our boundary condition
    
    x = np.full(1,a) #initial value
    y = np.full((1,nv), y_a) #initial condition
    
    flag = 1 #creating a flag variable for later loops
    
    while(flag):
        yi_new, h_new, h_step = rk4_mv_ad(dydx, xi, yi, nv, h, tol) #sending an answer, step attempt, and step function into our driver
        
        h = h_new #setting our step to our step attempt
        
        if(xi + h_step>b): #checking to see if we overshot
            
            h = b-xi #smaller step
            
            yi_new, h_new, h_step = rk4_mv_ad(dydx, xi, yi, nv, h, tol) #if we overshot, recalculating with a smaller step
            
            flag = 0 #breaking the function
            
        xi += h_step # updating our values
        yi[:] = yi_new[:]
        
        x = np.append(x, xi) #adding to our x array the new values
        
        y_new = np.zeros((len(x), nv) ) #adding to all of our arrays the updated functions
        
        y_new[0:len(x)-1,:] = y
        y_new[-1:] = yi[:]
        
        del y
        
        y = y_new
        
        if(i>=imax): #stopping if we hit max iterations
            print("max iterations reached")
            raise StopIterations("Iterations stopped: ",i)
            
        i+=1
        
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i, xi, h_step, b) #output information as we iterate through functions
        print(s)
        
        if(xi==b):
            flag = 0
    return x,y

In [ ]:
a = 0.0 #setting our initial values
b = 2.0*np.pi

y_0 = np.zeros(2) 
y_0[0] = 0.0
y_0[1] = 1.0

nv = 2

tolerance = 1.0e-6

x,y = rk4_mv(dydx,a,b,y_0,tolerance) #calling our function to perform the operation

# Plot the Result

In [ ]:
plt.plot(x,y[:,0], 'o',label='y(x)') #plotting the initial function
plt.plot(x,y[:,1], 'o', label='dydx(x)') #plotting the dydx function
xx = np.linspace(0,2.0*np.pi,1000) #plotting trendlines
plt.plot(xx,np.sin(xx),label = 'sin(x)')
plt.plot(xx,np.cos(xx),label = 'cos(x)')
plt.xlabel('x') #labeling
plt.ylabel('y, dydx')
plt.legend(frameon=False)

In [ ]:
sine = np.sin(x) #creating an error plot
cosine = np.cos(x)

y_error = (y[:,0]-sine)
dydx_error = (y[:,1]-cosine)

plt.plot(x, y_error, label = 'y(x) Error') #plotting the function error 
plt.plot(x, dydx_error, label='dydx(x) Error') #plotting the differential function error
plt.legend(frameon=False)